# Detección de anomalías

En ciencia de datos, la *detección de anomalías* es la identificación de observaciones raras, que se desvían significativamente de la "normalidad" definida en un conjunto de datos. 

![](https://i.imgur.com/vTIXxm8m.jpg)


Entendida como **un paso en el análisis exploratorio de datos o EDA**, podemos buscar anomalías considerándolas ruido en los datos que puede ensuciar la muestra y deteriorar la capacidad de un clasificador o un regresor. En estos escenarios hablamos de detección y limpieza de _outliers_.

La detección de anomalías es a veces también **un fin en sí misma**. En estos casos, no consideramos "ruido" a los datos atípicos ni buscamos desecharlos: al contrario, identificarlos resulta del mayor valor. Por ejemplo, en un entorno médico podría ser de interés detectar muestras de sangre o imágenes anómalas. Otro ejemplo podría ser los sensores de una turbina de un avión: la detección de una anomalía podría ser de enorme relevancia en estos casos.

Si bien existen otros enfoques, la detección de anomalías suele encararse como un problema de **Aprendizaje Automático No-Supervisado**. En este enfoque, se ajusta un modelo a los datos, pero no hay etiquetas `y`, solamente la `X`. Esto se debe a que, por definición, las anomalías son escasas y no es fácil conseguir un dataset con muchas de ellas. 


Existen diferentes formas de ajustar modelos. Podemos pensar que se busca construir una cierta idea de "normalidad" a partir de los datos para, desde esa idea, identificar los datos que se apartan de ella como "anomalías".


# Detección de anomalías en un dataset de cáncer de mama

Vamos a usar un dataset público de mamografías bajado de [ODDS](http://odds.cs.stonybrook.edu/mammography-dataset/). Este consiste de `11183` mamografías con `6` caracteristicas (anonimizadas). De estas, solo `260` corresponden a tumores malignos.

Entrenaremos un modelo de detección de anomalías sin considerar la etiqueta y veremos cuán bien funciona esta detección de anomalías para identificar tumores malignos.

In [1]:
# Importamos las librerías relevantes
import numpy as np
import pandas as pd
import arff

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, recall_score
import warnings 
warnings.simplefilter('ignore')

In [8]:
with open('data/cancer.arff', 'r') as file:
    data = arff.load(file)

# Convertir los datos a un DataFrame de Pandas
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])

# Transformar class en numérico y renombrarlo
df['class'] = df['class'].apply(lambda row: 0 if row =='-1' else 1)
df = df.rename(columns={'class':'es_cancer'})

# Mostrar los primeros registros del DataFrame
df.head()

,attr1,attr2,attr3,attr4,attr5,attr6,es_cancer
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [9]:
# 6 features anonimizados
print(df.shape)
df.head()

(11183, 7)


,attr1,attr2,attr3,attr4,attr5,attr6,es_cancer
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [10]:
# Solo 260 datos son tumores malignos
df['es_cancer'].value_counts()

es_cancer
0    10923
1      260
Name: count, dtype: int64

In [11]:
# Corresponde al 2.3% de los datos
df['es_cancer'].value_counts(normalize=True)

es_cancer
0    0.97675
1    0.02325
Name: proportion, dtype: float64

## Entrenamos un modelo [`IsolationForest`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest)

Isolation Forest es un algoritmo de detección de anomalias. Este se basa en construir un árbol de decisión sobre los datos (varios, en realidad) y medir cuantos cortes fueron necesarios para _aislar_ un punto (de ahí su nombre). 
Esta técnica se diferencia de la mayoría de los modelos de detección de anomalías que construyen una representación de "normalidad" para luego ver cuánto se aparta un punto de esta.

<img src='https://i.imgur.com/LkIceyM.png' style="width:800px;height:300px">

Separemos el set de datos en datos de entrenamiento y validación de la forma usual:

In [12]:
X, y = df.drop('es_cancer',axis=1), df['es_cancer']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
contamination = y_train.value_counts(normalize=True)[1]
contamination

np.float64(0.023250268272326218)

In [14]:
y_test.value_counts()

es_cancer
0    2731
1      65
Name: count, dtype: int64

In [15]:
model = IsolationForest(contamination='auto', random_state=1)

In [16]:
# El método fit no usa y_train!! Está aprendiendo a detectar anomalías, no etiquetas o clases
model.fit(X_train)

IsolationForest(random_state=1)

El método `predict` de un modelo de detección de anomalías retorna `1` (outlier) o `-1` (normal):

```python
Signature: model.predict(X)
Docstring:
Predict if a particular sample is an outlier or not.

Parameters
----------
X : {array-like, sparse matrix} of shape (n_samples, n_features)
    The input samples. Internally, it will be converted to
    ``dtype=np.float32`` and if a sparse matrix is provided
    to a sparse ``csr_matrix``.

Returns
-------
is_inlier : ndarray of shape (n_samples,)
    For each observation, tells whether or not (+1 or -1) it should
    be considered as an inlier according to the fitted model.
```

In [17]:
y_pred = model.predict(X_test)
y_pred = [1 if y == -1 else 0 for y in y_pred]

Veamos la matriz de confusión y el reporte de clasificación para este detector:

In [18]:
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns=['Pred=Sano', 'Pred=Cancer'], index=['Actual=Sano', 'Actual=Cancer'])
cm

,Pred=Sano,Pred=Cancer
Actual=Sano,2401,330
Actual=Cancer,22,43


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.88      0.93      2731
           1       0.12      0.66      0.20        65

    accuracy                           0.87      2796
   macro avg       0.55      0.77      0.56      2796
weighted avg       0.97      0.87      0.91      2796



Obtuvimos un `66%` de exhaustividad en identificación de muestras cancerosas con este sencillo detector de anomalías.

Veremos la tasa de falsos positivos, que funciona como un "trade-off" con la exhaustividad. Con esto, podemos aumentar la exhaustividad aumentando la tasa de falsos positivos.


In [20]:
recall_score(y_test, y_pred)

np.float64(0.6615384615384615)

In [21]:
# Ratio de Falsos Positivos False positive rate FPR
fp = cm.loc['Actual=Sano', 'Pred=Cancer']
p  = cm.loc['Actual=Sano'].sum()
fpr = fp / p
fpr

np.float64(0.1208348590259978)

La tasa de falsos positivos es de 12%. 
Vamos a probar diferentes valores del parámetro "contamination" para ver cómo se mueven la exhaustividad y la tasa de falsos positivos, y ver si encontramos alguna combinación que nos guste más...

In [22]:
def get_results_for_factor(X_train, y_train, X_test, y_test, contamination_factor, verbose=False):
    contamination = y_train.value_counts(normalize=True)[1]
    C = contamination_factor*contamination
    model = IsolationForest(contamination=C, random_state=1)
    model.fit(X_train)
    y_pred = model.predict(X_test)
    y_pred = [1 if y == -1 else 0 for y in y_pred]
    
    cm = confusion_matrix(y_test, y_pred)
    cm = pd.DataFrame(cm, columns=['Pred=Sano', 'Pred=Cancer'], index=['Actual=Sano', 'Actual=Cancer'])
    fp = cm.loc['Actual=Sano', 'Pred=Cancer']
    p  = cm.loc['Actual=Sano'].sum()
    fpr = fp / p
    recall = recall_score(y_test, y_pred)
    results = {'contamination_factor': contamination_factor, 'recall': round(recall, 2), 'fpr': round(fpr, 2)}
    
    if verbose:
        print(results)
        print(classification_report(y_test, y_pred))
        display(cm)
    return results

In [23]:
all_res = []
for factor in range(1, 20):
    res = get_results_for_factor(X_train, y_train, X_test, y_test, factor)
    print(res)
    all_res.append(res)

{'contamination_factor': 1, 'recall': np.float64(0.22), 'fpr': np.float64(0.01)}
{'contamination_factor': 2, 'recall': np.float64(0.32), 'fpr': np.float64(0.03)}
{'contamination_factor': 3, 'recall': np.float64(0.49), 'fpr': np.float64(0.06)}
{'contamination_factor': 4, 'recall': np.float64(0.54), 'fpr': np.float64(0.08)}
{'contamination_factor': 5, 'recall': np.float64(0.62), 'fpr': np.float64(0.11)}
{'contamination_factor': 6, 'recall': np.float64(0.72), 'fpr': np.float64(0.13)}
{'contamination_factor': 7, 'recall': np.float64(0.8), 'fpr': np.float64(0.15)}
{'contamination_factor': 8, 'recall': np.float64(0.82), 'fpr': np.float64(0.19)}
{'contamination_factor': 9, 'recall': np.float64(0.82), 'fpr': np.float64(0.21)}
{'contamination_factor': 10, 'recall': np.float64(0.83), 'fpr': np.float64(0.24)}
{'contamination_factor': 11, 'recall': np.float64(0.86), 'fpr': np.float64(0.26)}
{'contamination_factor': 12, 'recall': np.float64(0.91), 'fpr': np.float64(0.28)}
{'contamination_factor': 1

### 80% de exhaustividad (recall) con solo 15% de falsos positivos!!

In [24]:
res = get_results_for_factor(X_train, y_train, X_test, y_test, 7, verbose=True)

{'contamination_factor': 7, 'recall': np.float64(0.8), 'fpr': np.float64(0.15)}
              precision    recall  f1-score   support

           0       0.99      0.85      0.91      2731
           1       0.11      0.80      0.19        65

    accuracy                           0.84      2796
   macro avg       0.55      0.82      0.55      2796
weighted avg       0.97      0.84      0.90      2796



,Pred=Sano,Pred=Cancer
Actual=Sano,2310,421
Actual=Cancer,13,52
